# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# confirm that our new database was created
collections = db.list_collection_names()
print("Collections in 'uk_food' database:", collections)

# Check if the 'establishments' collection exists
if 'establishments' in collections:
    print("\nSample document from 'establishments' collection:")
    pprint(db.establishments.find_one())  # Print one sample document
else:
    print("\n'establishments' collection not found in the database.")

Collections in 'uk_food' database: ['establishments']

Sample document from 'establishments' collection:
{'AddressLine1': '123 Food Street',
 'AddressLine2': 'London',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'Latitude': 51.515,
 'Longitude': -0.09,
 'PostCode': 'EC1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('67c66f61b4b8db15a2307b18')}


In [8]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
# review the collections in our new database
##twice?

In [10]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': '123 Food Street',
 'AddressLine2': 'London',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'Latitude': 51.515,
 'Longitude': -0.09,
 'PostCode': 'EC1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('67c66f61b4b8db15a2307b18')}


In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [14]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Nando's",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "43 Churchill Street",
    "AddressLine2": "London",
    "PostCode": "EC1A 1AA",
    "RatingValue": 4,
    "Latitude": 51.515,
    "Longitude": -0.09
}

In [15]:
# Insert the new restaurant into the collection
db.establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('67c671e619f176ade9467239'), acknowledged=True)

In [16]:
# Check that the new restaurant was inserted
pprint(db.establishments.find_one({"BusinessName": "Nando's"}))


{'AddressLine1': '43 Churchill Street',
 'AddressLine2': 'London',
 'BusinessName': "Nando's",
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'Latitude': 51.515,
 'Longitude': -0.09,
 'PostCode': 'EC1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('67c671e619f176ade9467239')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [18]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
business_type = db.establishments.find_one(query, {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})
pprint(business_type)


{'BusinessType': 'Restaurant/Cafe/Canteen'}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [20]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = business_type["BusinessType"]

restaurant_filter = {"BusinessName": "Lou Manati's"} 

update_query = {"$set": {"BusinessTypeID": business_type_id}}

db.establishments.update_one(restaurant_filter, update_query)

# Verify the update
updated_restaurant = db.establishments.find_one(restaurant_filter)
pprint(updated_restaurant)

None


In [21]:
# Retrieve a single document to inspect field names
document_sample = db.establishments.find_one()
pprint(document_sample)


{'AddressLine1': '123 Food Street',
 'AddressLine2': 'London',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'Latitude': 51.515,
 'Longitude': -0.09,
 'PostCode': 'EC1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('67c66f61b4b8db15a2307b18')}


In [22]:
# Confirm that the new restaurant was updated
updated_restaurant = db.establishments.find_one({"BusinessName": "New Restaurant Name"})
pprint(updated_restaurant)

None


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [24]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of establishments in Dover: {dover_count}")

Number of establishments in Dover: 0


In [25]:
# Delete all documents where LocalAuthorityName is "Dover"
result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})

print(f"Number of documents deleted: {result.deleted_count}")


Number of documents deleted: 0


In [58]:
# Check if any remaining documents include Dover
remaining_documents = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of remaining documents with LocalAuthorityName as 'Dover': {remaining_documents}")


Number of remaining documents with LocalAuthorityName as 'Dover': 0


In [60]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one()
pprint(remaining_document)


{'AddressLine1': '123 Food Street',
 'AddressLine2': 'London',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'Latitude': 51.515,
 'Longitude': -0.09,
 'PostCode': 'EC1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('67c66f61b4b8db15a2307b18')}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [62]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many(
    {"latitude": {"$exists": True}, "longitude": {"$exists": True}},  # Ensure fields exist
    [
        {"$set": {
            "latitude": {"$toDecimal": "$latitude"},
            "longitude": {"$toDecimal": "$longitude"}
        }}])

# Confirm data type change by retrieving one updated document
updated_doc = db.establishments.find_one({}, {"latitude": 1, "longitude": 1})
pprint(updated_doc)

{'_id': ObjectId('67c66f61b4b8db15a2307b18')}


Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [33]:
# Change the data type from String to Integer for RatingValue


In [34]:
# Check that the coordinates and rating value are now numbers
